In [1]:
%load_ext autoreload
%autoreload 2

loading file
[Source](http://files.fast.ai/models/wt103/)

In [2]:
import os
import pickle
import datetime
import pandas as pd
import torch
import torch.nn as nn

from training_utils import training_loop, test_loop
from model_v2 import RNNLM
# from data_utils import IndexVectorizer, TextDataset, simple_tokenizer

## Downloading pre-trained weights

In [3]:
directory_pre_trained_models = '/scratch/olympus/projects/sentiment_analysis/machine_learnining/weights_pretrained'
os.makedirs(directory_pre_trained_models, exist_ok=True)

In [4]:
# !wget http://files.fast.ai/models/wt103/fwd_wt103_enc.h5 -O $directory_pre_trained_models/fwd_wt103_enc.h5

Can skip this one V

In [5]:
# !wget http://files.fast.ai/models/wt103/fwd_wt103.h5 -O $directory_pre_trained_models/fwd_wt103.h5

In [6]:
# !wget http://files.fast.ai/models/wt103/itos_wt103.pkl -O $directory_pre_trained_models/fitos_wt103.pkl

In [11]:
!ls -ltha $directory_pre_trained_models

total 886M
drwxrws---+ 2 ly501 smapp 4.0K Dec  4 17:15 .
drwxrws---+ 3 ly501 smapp 4.0K Dec  4 17:11 ..
-rwxrwx---+ 1 ly501 smapp 441M Mar 28  2018 fwd_wt103.h5
-rwxrwx---+ 1 ly501 smapp 441M Mar 28  2018 fwd_wt103_enc.h5
-rwxrwx---+ 1 ly501 smapp 4.0M Mar 28  2018 fitos_wt103.pkl


## Loading weights

In [7]:
encoder_file = os.path.join(directory_pre_trained_models, 'fwd_wt103.h5')
weights_file = os.path.join(directory_pre_trained_models, 'fwd_wt103_enc.h5')
fitos_file = os.path.join(directory_pre_trained_models, 'fitos_wt103.pkl')

In [8]:
wgts = torch.load(weights_file, map_location=lambda storage, loc: storage)

In [9]:
enc = torch.load(encoder_file, map_location=lambda storage, loc: storage)

In [10]:
for k,v in enc.items():
    print(f'{k}:    {v.size()}')

0.encoder.weight:    torch.Size([238462, 400])
0.encoder_with_dropout.embed.weight:    torch.Size([238462, 400])
0.rnns.0.module.weight_ih_l0:    torch.Size([4600, 400])
0.rnns.0.module.bias_ih_l0:    torch.Size([4600])
0.rnns.0.module.bias_hh_l0:    torch.Size([4600])
0.rnns.0.module.weight_hh_l0_raw:    torch.Size([4600, 1150])
0.rnns.1.module.weight_ih_l0:    torch.Size([4600, 1150])
0.rnns.1.module.bias_ih_l0:    torch.Size([4600])
0.rnns.1.module.bias_hh_l0:    torch.Size([4600])
0.rnns.1.module.weight_hh_l0_raw:    torch.Size([4600, 1150])
0.rnns.2.module.weight_ih_l0:    torch.Size([1600, 1150])
0.rnns.2.module.bias_ih_l0:    torch.Size([1600])
0.rnns.2.module.bias_hh_l0:    torch.Size([1600])
0.rnns.2.module.weight_hh_l0_raw:    torch.Size([1600, 400])
1.decoder.weight:    torch.Size([238462, 400])


In [11]:
# they're the same vals
wgts['rnns.2.module.bias_hh_l0'] == enc['0.rnns.2.module.bias_hh_l0']

tensor([1, 1, 1,  ..., 1, 1, 1], dtype=torch.uint8)

In [12]:
# GPU variables
use_gpu = torch.cuda.is_available()
device_num = 0
device = torch.device(f"cuda:{device_num}" if use_gpu else "cpu")
device

device(type='cuda', index=0)

In [13]:
vocab_size = 238462
embedding_size = 400
hidden_size = 4600
num_layers = 3

In [14]:
# example of a model
lm = RNNLM(device, vocab_size, embedding_size, hidden_size, 256, num_layers=3)

In [17]:
layer_detail

['0', 'encoder', 'weight']

In [18]:
# load weights
for k, v in enc.items():    
    layer_detail = k.split('.')
    layer_name = layer_detail[-1].replace('_raw', '')
    
    print(layer_detail)

    try:
        layer = getattr(lm, layer_detail[1])
    except Exception as e:
        print(e)
        
    # which rnn, and what layer???
    if layer_detail[1] == 'rnns':
        n_rnn = int(layer_detail[2])
        layer = layer[n_rnn]
    try:
        # this is what assigns the new value
        getattr(layer, layer_name).data = v
    
    except Exception as e:
        print(e)(layer_detail)

['0', 'encoder', 'weight']
['0', 'encoder_with_dropout', 'embed', 'weight']
'RNNLM' object has no attribute 'encoder_with_dropout'
['0', 'rnns', '0', 'module', 'weight_ih_l0']
['0', 'rnns', '0', 'module', 'bias_ih_l0']
['0', 'rnns', '0', 'module', 'bias_hh_l0']
['0', 'rnns', '0', 'module', 'weight_hh_l0_raw']
['0', 'rnns', '1', 'module', 'weight_ih_l0']
['0', 'rnns', '1', 'module', 'bias_ih_l0']
['0', 'rnns', '1', 'module', 'bias_hh_l0']
['0', 'rnns', '1', 'module', 'weight_hh_l0_raw']
['0', 'rnns', '2', 'module', 'weight_ih_l0']
['0', 'rnns', '2', 'module', 'bias_ih_l0']
['0', 'rnns', '2', 'module', 'bias_hh_l0']
['0', 'rnns', '2', 'module', 'weight_hh_l0_raw']
['1', 'decoder', 'weight']
